In [26]:
#Taller #10: Twitter
#Fecha de entrega: Noviembre 14, 2020 (Antes del inicio de la próxima clase).

#Modo de entrega: Subir link de GitHub al aula virtual.

#[25 pts] Punto 1: Extraer Tuits
#Extraer mínimo 100 tuits de Twitter usando la API

#(Si exponen sus claves, les haré maldades 😈)

#❌🎃❌ ¡No Halloween!

In [27]:
import pandas as pd
import tweepy
import re
import regex
import emoji


In [28]:
# Leer llaves
keys = pd.read_csv("keys.csv", header=None) #se deja con la etiqueta None ya que no tiene titulos el archivo
keys = dict(zip(keys[0],keys[1])) #se deja en modo diccionario


In [29]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['API_key'], keys['API_key_secret'])
auth.set_access_token(keys['acces_token'], keys['acces_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [55]:
#Recolectar información: Tweets de la cuidad de Bogotá
tweets = tweepy.Cursor(api.search,
                       q=['Bancamia -RT'],
                       lang='es',
                       geocode=f'4.60971,-74.08175,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(500)

In [56]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogota")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)

In [57]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,Bancamía; trabaja en alianza con la Fundación ...,2020-11-13 18:39:02,Bogota,villavicencio,1327320079902547968,https://twitter.com/i/web/status/1327320079902...
1,@LaObando10 @Bancompartir Lo mismo sucede en B...,2020-11-12 18:20:56,Bogota,"Madrid, Colombia",1326953137899507715,https://twitter.com/i/web/status/1326953137899...
2,"Bancamía tiene 1.3 millones de clientes, de lo...",2020-11-12 10:55:02,Bogota,"Bogotá, Colombia",1326840925331595264,https://twitter.com/i/web/status/1326840925331...
3,"Bancamía, entidad del Grupo Fundación Microfin...",2020-11-11 21:03:37,Bogota,villavicencio,1326631692216496128,https://twitter.com/i/web/status/1326631692216...
4,Bancamía recibe reconocimiento por su trabajo ...,2020-11-11 20:16:59,Bogota,"Bogotá, Colombia",1326619956704841730,https://twitter.com/i/web/status/1326619956704...


In [58]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,Bancamía; trabaja en alianza con la Fundación ...,2020-11-13 18:39:02,Bogota
1,@LaObando10 @Bancompartir Lo mismo sucede en B...,2020-11-12 18:20:56,Bogota
2,"Bancamía tiene 1.3 millones de clientes, de lo...",2020-11-12 10:55:02,Bogota
3,"Bancamía, entidad del Grupo Fundación Microfin...",2020-11-11 21:03:37,Bogota
4,Bancamía recibe reconocimiento por su trabajo ...,2020-11-11 20:16:59,Bogota


In [59]:
# Realizamos el Preprocesamiento de la data: Se depurar emojis, hashtags, hipervinculos

import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [60]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data.head()

,tweet,date,city,clean,links,hashtags,mentions
0,Bancamía; trabaja en alianza con la Fundación ...,2020-11-13 18:39:02,Bogota,Bancamía; trabaja en alianza con la Fundación ...,[https://t.co/zTdbas7gvd],[],[]
1,@LaObando10 @Bancompartir Lo mismo sucede en B...,2020-11-12 18:20:56,Bogota,Lo mismo sucede en Bancamia; durante la pand...,[],[],"[@LaObando10, @Bancompartir]"
2,"Bancamía tiene 1.3 millones de clientes, de lo...",2020-11-12 10:55:02,Bogota,"Bancamía tiene 1.3 millones de clientes, de lo...",[https://t.co/rgKdL54kxI],[],[]
3,"Bancamía, entidad del Grupo Fundación Microfin...",2020-11-11 21:03:37,Bogota,"Bancamía, entidad del Grupo Fundación Microfin...",[https://t.co/SHCdXsJOMC],[],[]
4,Bancamía recibe reconocimiento por su trabajo ...,2020-11-11 20:16:59,Bogota,Bancamía recibe reconocimiento por su trabajo ...,[https://t.co/a9U4CNs98I],[],[]


In [61]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis
0,Bancamía; trabaja en alianza con la Fundación ...,2020-11-13 18:39:02,Bogota,Bancamía; trabaja en alianza con la Fundación ...,[https://t.co/zTdbas7gvd],[],[],[]
1,@LaObando10 @Bancompartir Lo mismo sucede en B...,2020-11-12 18:20:56,Bogota,Lo mismo sucede en Bancamia; durante la pand...,[],[],"[@LaObando10, @Bancompartir]",[]
2,"Bancamía tiene 1.3 millones de clientes, de lo...",2020-11-12 10:55:02,Bogota,"Bancamía tiene 1.3 millones de clientes, de lo...",[https://t.co/rgKdL54kxI],[],[],[]
3,"Bancamía, entidad del Grupo Fundación Microfin...",2020-11-11 21:03:37,Bogota,"Bancamía, entidad del Grupo Fundación Microfin...",[https://t.co/SHCdXsJOMC],[],[],[]
4,Bancamía recibe reconocimiento por su trabajo ...,2020-11-11 20:16:59,Bogota,Bancamía recibe reconocimiento por su trabajo ...,[https://t.co/a9U4CNs98I],[],[],[]


In [62]:
#[25 pts] Punto 2: Hacer una visualización sobre esos tuits
#❌🎃❌ ¡No Halloween!

In [63]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
2,2020-11-06,2
0,2020-11-11,3
1,2020-11-12,2
3,2020-11-13,1


In [64]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [67]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'Red'})


layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [66]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
0,🙋🏽‍♀️,1
1,💁‍♂️,1
2,➡️,1


In [68]:
trace = go.Bar(x=temp_emojis.emoji.values,
               y=temp_emojis.cnt.values)

layout = go.Layout(title='Emojis más usados para hablar de Bancamía')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)